In [6]:
from tinygrad import Device
print(Device.DEFAULT)

CPU


In [24]:
from tinygrad import Tensor, nn

class Model:
    def __init__(self):
        self.l1 = nn.Conv2d(1, 32, kernel_size=(3,3))
        self.l2 = nn.Conv2d(32,64,kernel_size=(3,3))
        self.l3 = nn.Linear(1600, 10)

    def __call__(self, x:Tensor) -> Tensor:
        x = self.l1(x).relu().max_pool2d((2,2))
        x = self.l2(x).relu().max_pool2d((2,2))
        return self.l3(x.flatten(1).dropout(0.5))

In [11]:
from tinygrad.nn.datasets import mnist
X_train, Y_train, X_test, Y_test = mnist()
print(X_train.shape, X_train.dtype, Y_train.shape, Y_train.dtype)

(60000, 1, 28, 28) dtypes.uchar (60000,) dtypes.uchar


In [25]:
model = Model()
acc = (model(X_test).argmax(axis=1) == Y_test).mean()
# NOTE: tinygrad is lazy, and hasn't actually run anything by this point
print(acc.item())  # ~10% accuracy, as expected from a random model

0.07479999959468842


In [26]:
optim = nn.optim.Adam(nn.state.get_parameters(model))
batch_size = 128
def step():
  Tensor.training = True  # makes dropout work
  samples = Tensor.randint(batch_size, high=X_train.shape[0])
  X, Y = X_train[samples], Y_train[samples]
  optim.zero_grad()
  loss = model(X).sparse_categorical_crossentropy(Y).backward()
  optim.step()
  return loss

In [27]:
import timeit
timeit.repeat(step, repeat=5, number=1)
#[0.08268719699981375,
# 0.07478952900009972,
# 0.07714716600003158,
# 0.07785399599970333,
# 0.07605237000007037]

[3.9447798229994078,
 1.2505387560013332,
 0.8776281890022801,
 0.8732907150006213,
 0.9003348250007548]

In [28]:
from tinygrad import TinyJit
jit_step = TinyJit(step)

In [32]:
import timeit
timeit.repeat(jit_step, repeat=5, number=1)
# [0.2596786549997887,
#  0.08989566299987928,
#  0.0012115650001760514,
#  0.001010227999813651,
#  0.0012164899999334011]

[0.9349846709992562,
 0.8772227720000956,
 0.8776176799983659,
 0.8782093999980134,
 0.8755927239981247]

In [33]:
for step in range(7000):
  loss = jit_step()
  if step%100 == 0:
    Tensor.training = False
    acc = (model(X_test).argmax(axis=1) == Y_test).mean().item()
    print(f"step {step:4d}, loss {loss.item():.2f}, acc {acc*100.:.2f}%")

step    0, loss 1.43, acc 78.71%
step  100, loss 0.34, acc 94.34%
step  200, loss 0.19, acc 95.59%
step  300, loss 0.24, acc 96.66%
step  400, loss 0.22, acc 97.16%


KeyboardInterrupt: 

In [7]:
from tinygrad import Tensor

a = Tensor.empty(4,4)
b = Tensor.empty(4,4)
print((a+b))

<Tensor <UOp CPU (4, 4) float (<Ops.ADD: 48>, None)> on CPU with grad None>
